In [16]:
import pandas as pd
from pandas.tseries.offsets import *
from sklearn import preprocessing
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import math
from copy import deepcopy
import json
from itertools import combinations, permutations
import warnings

% matplotlib inline
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 500)
warnings.filterwarnings('ignore')

In [17]:
def teacher_qc_score(start_time, end_time, df_qc, class_type_name_special, hq_name_special):
    # 筛选条件
    df_qc = df_qc.loc[(~df_qc['class_type_name'].isin(class_type_name_special)) & 
                    (~df_qc['hq_name'].isin(hq_name_special)) & 
                    (df_qc['start_time'] >= start_time) & (df_qc['end_time'] <= end_time)]
    # log score-mean
    df_qc['decay_index'] = ((end_time - df_qc['score_recorded_at']).dt.total_seconds()) / (3600 * 24) 
    df_qc.loc[df_qc['decay_index'] < 0, 'decay_index'] = 0 # qc时间有时会在end_time之后造成负值
    df_qc['decay_index'] = df_qc['decay_index'] + 2 # log计算，防止0值+1;取倒数+1，防止无穷情况
    df_qc['decay_index'] = df_qc['decay_index'].apply(lambda x: math.log(x, 10))
    df_qc['decay_index'] = 1 / df_qc['decay_index']
    df_qc['log_decay_score'] = df_qc['score'] * df_qc['decay_index']
    df_teacher_log_score = df_qc.groupby(['awj_teacher_id'], 
                        as_index=False).agg({'log_decay_score': np.sum, 'decay_index': np.sum})
    df_teacher_log_score['log_decay_score_mean'] = df_teacher_log_score['log_decay_score'] / df_teacher_log_score['decay_index']
    df_teacher_log_score = df_teacher_log_score[['awj_teacher_id', 'log_decay_score_mean']]
    # score
    df_teacher_score = df_qc.groupby(['awj_teacher_id'], 
                as_index=False)['score'].agg(['max', 'min', 'count', np.std]) 
    # std missing
    df_teacher_score.loc[df_teacher_score['std'].isnull(), 'std'] = df_teacher_score['std'].mean()
    df_teacher_score.rename(columns={'max': 'teacher_score_max', 'min': 'teacher_score_min', 
                        'count': 'teacher_qc_count', 'std': 'teacher_score_std'}, inplace=True)
    df_teacher_score.reset_index(inplace=True)
    # merge
    df_qc_res = pd.merge(df_teacher_score, df_teacher_log_score, on='awj_teacher_id', how='left')
    return df_qc_res

In [18]:
def teacher_behavior(start_time, end_time, df_teacher_behavior, 
                                class_type_name_special, hq_name_special):
    # 筛选条件
    df_teacher_behavior = df_teacher_behavior.loc[
                    (~df_teacher_behavior['上课类型'].isin(class_type_name_special)) & 
                    (~df_teacher_behavior['机构'].isin(hq_name_special)) & 
                    (df_teacher_behavior['start_time'] >= start_time) & 
                    (df_teacher_behavior['end_time'] <= end_time) & 
                    (~df_teacher_behavior['teacher_status_for_lesson'].isin(['system_failure']))]
    # 预处理
    df_teacher_behavior['lesson_count'] = ((df_teacher_behavior['end_time'] - 
                        df_teacher_behavior['start_time']).dt.total_seconds()) / 3600 * 2
    df_teacher_behavior.loc[df_teacher_behavior['teacher_status_for_lesson'] == 'no_show', 
                            'ask_for_leave_advanced_minutes'] = 0
    df_teacher_behavior['ask_for_leave_advanced_days'] = df_teacher_behavior['ask_for_leave_advanced_minutes'] / (60 * 24)
    # 天数衰减
    df_teacher_behavior['decay_index'] = ((end_time - 
            df_teacher_behavior['start_time']).dt.total_seconds()) / (3600 * 24) 
    df_teacher_behavior.loc[df_teacher_behavior['decay_index'] < 0, 'decay_index'] = 0
    # log计算，防止0值+1;取倒数+1，防止无穷情况
    df_teacher_behavior['decay_index'] = df_teacher_behavior['decay_index'] + 2
    df_teacher_behavior['decay_index'] = df_teacher_behavior['decay_index'].apply(lambda x: math.log(x, 10))
    df_teacher_behavior['decay_index'] = 1 / df_teacher_behavior['decay_index']
    # count lesson types
    df_teacher_behavior_res = df_teacher_behavior[['awj_teacher_id']]
    df_teacher_behavior_res.drop_duplicates(keep='first', inplace=True)
    types = ['normal_lesson', 'late', 'no_show', 'abnormal_lesson', 'ask_for_leave']
    for itm in types:
        df_count = df_teacher_behavior.loc[
            df_teacher_behavior['teacher_status_for_lesson'].isin([itm])]
        df_count['log_lesson_count'] = df_count['lesson_count'] * df_count['decay_index']
        df_count = df_count.groupby(['awj_teacher_id'], as_index=False)['log_lesson_count'].sum()
        df_count.reset_index()
        if (itm.find('ask_for_leave') > -1) | (itm.find('lesson') > -1):
            df_count.rename(columns={'log_lesson_count': itm + '_log_count'}, inplace=True)
        else:
            df_count.rename(columns={'log_lesson_count': itm + '_lesson_log_count'}, inplace=True)
        df_teacher_behavior_res = pd.merge(df_teacher_behavior_res, df_count, 
                                                  on='awj_teacher_id', how='left')
    # ask for leave advanced minutes
    df_advanced_days = df_teacher_behavior.loc[
        df_teacher_behavior['teacher_status_for_lesson'].isin(['ask_for_leave', 'no_show'])]  
    df_advanced_days['ask_for_leave_advanced_log_days'] = df_advanced_days['decay_index'] * df_advanced_days['ask_for_leave_advanced_days']
    # 求均值
    df_advanced_days_mean = df_advanced_days.groupby(['awj_teacher_id'], as_index=False).agg(
        {'ask_for_leave_advanced_log_days': np.sum, 'decay_index': np.sum})
    df_advanced_days_mean.reset_index(inplace=True)
    df_advanced_days_mean['advanced_days_log_mean'] = df_advanced_days_mean['ask_for_leave_advanced_log_days'] / df_advanced_days_mean['decay_index'] 
    df_advanced_days_mean = df_advanced_days_mean[['awj_teacher_id', 'advanced_days_log_mean']]
    # 求最大最小值及方差
    df_advanced_days_others = df_advanced_days.groupby(['awj_teacher_id'], 
            as_index=False)['ask_for_leave_advanced_days'].agg(['min', 'max', 'std'])
    df_advanced_days_others.reset_index(inplace=True)
    df_advanced_days_others.loc[df_advanced_days_others['std'].isnull(), 
                                'std'] = df_advanced_days_others['std'].mean()
    df_advanced_days_others.rename(columns={'min': 'advanced_days_min', 
                    'max': 'advanced_days_max', 'std': 'advanced_days_std'}, inplace=True)
    df_advanced_days = pd.merge(df_advanced_days_mean, df_advanced_days_others, 
                               on='awj_teacher_id', how='left')
    df_teacher_behavior_res = pd.merge(df_teacher_behavior_res, df_advanced_days, 
                                                       on='awj_teacher_id', how='left')
    # fill 0
    columns = ['normal_lesson_log_count', 'late_lesson_log_count', 'no_show_lesson_log_count', 
               'abnormal_lesson_log_count', 'ask_for_leave_log_count', 'advanced_days_log_mean', 
               'advanced_days_min', 'advanced_days_max', 'advanced_days_std']
    for itm in columns:
        df_teacher_behavior_res[itm].fillna(value=0, inplace=True)
    return df_teacher_behavior_res, df_teacher_behavior

In [19]:
def stu_comment(start_time, end_time, df_stu_comment, 
                class_type_name_special, hq_name_special, df_teacher_behavior_processed_cache):
    # 筛选条件：时间筛选条件由于筛选后数量过少，所以采用end time前的全量数据
    df_stu_comment = df_stu_comment.loc[(df_stu_comment['课程开始时间'] >= start_time) & 
                                (df_stu_comment['课程结束时间'] <= end_time) & 
                                (~df_stu_comment['机构'].isin(hq_name_special)) & 
                                (~df_stu_comment['课程类型'].isin(class_type_name_special))]
    print('df_stu_comment_floater classes existed:', df_stu_comment.shape)
    # 与行为表对比，去除floater课程
    df_stu_comment = pd.merge(df_stu_comment, df_teacher_behavior_processed_cache, 
                             left_on=['awj_teacher_id', 'awjcls_lesson_id'], 
                             right_on=['awj_teacher_id', 'awj_lesson_id'], how='left')
    df_stu_comment = df_stu_comment.loc[~df_stu_comment['is_deleted'].isnull()]
    df_stu_comment.drop(['awj_lesson_id', 'is_deleted'], axis=1, inplace=True)
    print('df_stu_comment_floater classes deleted:', df_stu_comment.shape)
    # 天数衰减
    df_stu_comment['decay_index'] = ((end_time - 
            df_stu_comment['评价时间']).dt.total_seconds()) / (3600 * 24) 
    df_stu_comment.loc[df_stu_comment['decay_index'] < 0, 'decay_index'] = 0
    # log计算，防止0值+1;取倒数+1，防止无穷情况
    df_stu_comment['decay_index'] = df_stu_comment['decay_index'] + 2
    df_stu_comment['decay_index'] = df_stu_comment['decay_index'].apply(lambda x: math.log(x, 10))
    df_stu_comment['decay_index'] = 1 / df_stu_comment['decay_index']  
    # 4，5星统计(4，5星负面标签都是系统bug)
    df_good_label = df_stu_comment.loc[df_stu_comment['学生评价星级'].isin(['5-star', '4-star'])]
    df_good_label['lesson_count_processed'] = df_good_label['lesson_count'] * df_good_label['decay_index']
    df_good_label = df_good_label.groupby(['awj_teacher_id'], 
                            as_index=False)['lesson_count_processed'].sum()
    df_good_label.rename(columns={'lesson_count_processed': 'stu_comment_log_good_behavior'}, inplace=True)
    # 1~3星统计
    df_bad_label = df_stu_comment.loc[~df_stu_comment['学生评价星级'].isin(['5-star', '4-star'])]
    df_bad_label['lesson_count_processed'] = df_bad_label['lesson_count'] * df_bad_label['decay_index']
    df_bad_label = df_bad_label.groupby(['awj_teacher_id'], 
                                as_index=False)['lesson_count_processed'].sum()
    df_bad_label.rename(columns={'lesson_count_processed': 'stu_comment_log_bad_behavior'}, inplace=True)
    # merge
    df_stu_comment_res = pd.merge(df_good_label, df_bad_label, on='awj_teacher_id', how='outer')
    df_stu_comment_res['stu_comment_log_bad_behavior'].fillna(value=0, inplace=True)
    df_stu_comment_res['stu_comment_log_good_behavior'].fillna(value=0, inplace=True)
    return df_stu_comment_res, df_stu_comment

In [20]:
if __name__ == '__main__':
    path = './FA_Model_data/'
    end_time = pd.to_datetime(datetime.datetime.now().date()) + MonthEnd(n=-1) + DateOffset(hours=23.99999)
    end_time = datetime.datetime(2018, 4, 30, 23, 59, 59)
    start_time = end_time - DateOffset(months=3)
    class_type_name_special = ['Demo', '补课(非爱乐奇直属老师)', '托福班（30刀）', 'TOFEL', 
                              'VIP Writing/TOFEL（35刀）', 'Elite Pilot', 'Feeback Session', 
                              'New Teacher Test Class', '补课(爱乐奇直属老师)', 'Test Class', 
                              'Academic Meeting (Long)', 'Cur Experience Session-S', 
                              'Training-receiving', 'Cur Experience Session-L', 
                              'VIP Writing/TOFEL', 'Orientation Class', 'Academic Meeting', 
                              'Experience-receiving']
    hq_name_special = ['test']
    print('start_time:', start_time, '\n', 'end_time:', end_time, '\n')
    
    ##### 老师监控表 #####
    df_teacher_monitoring = pd.read_csv(path + 'awj_teacher_monitoring.csv', encoding='utf-8', sep=',')
    # dtypes
    df_teacher_monitoring['awj_teacher_id'] = df_teacher_monitoring['awj_teacher_id'].astype('int')
    df_teacher_monitoring['created_at'] = pd.to_datetime(df_teacher_monitoring['created_at'])
    # drop duplicates
    df_teacher_monitoring.drop_duplicates(
                        subset=list(df_teacher_monitoring.columns), keep='first', inplace=True)
    # sort
    df_teacher_monitoring = df_teacher_monitoring.sort_values(
                    by=['awj_teacher_id', 'awjcls_lesson_id', 'created_at'], ascending=[1, 1, 1])
    # 该表只取abnormal_type为4，5的行，分别表示zoom崩溃和课件崩溃，非老师原因
    df_teacher_monitoring = df_teacher_monitoring.loc[
                    df_teacher_monitoring['abnormal_type'].isin([4, 5])]
    df_teacher_monitoring = df_teacher_monitoring.groupby(
                ['awj_teacher_id', 'awjcls_lesson_id', 'abnormal_type'], as_index=False).last()
    # 唯一主键awj_teacher_id+awjcls_lesson_id check
    cache = df_teacher_monitoring.groupby(['awj_teacher_id', 'awjcls_lesson_id'], 
                                      as_index=False)['abnormal_type'].count()
    print('df_teacher_monitoring key check_duplicate no(awj_teacher_id, awjcls_lesson_id):', 
          cache.loc[cache['abnormal_type'] > 1].shape[0], '\n')
    
    ##### 老师QC明细表 #####
    df_qc = pd.read_csv(path + '老师qc明细.csv', encoding='utf-8', sep=',')
    # dtypes
    df_qc['awj_teacher_id'] = df_qc['awj_teacher_id'].astype('int')
    df_qc['score_recorded_at'] = pd.to_datetime(df_qc['score_recorded_at'])
    df_qc['assigned_at'] = pd.to_datetime(df_qc['assigned_at'])
    df_qc['start_time'] = pd.to_datetime(df_qc['start_time'])
    df_qc['end_time'] = pd.to_datetime(df_qc['end_time'])
    print('df_qc_before drop duplicates:', df_qc.shape)
    # drop duplicates-老师同一堂课有时会有多次qc，check后发现分数都一样，所以去重时按照以下字段去重即可
    columns = ['awj_teacher_id', 'awjcls_lesson_id', 'score']
    df_qc.drop_duplicates(subset=columns, keep='last', inplace=True)
    print('df_qc_drop duplicates:', df_qc.shape)
    # sort
    df_qc = df_qc.sort_values(by=['awj_teacher_id', 'start_time'], ascending=[1, 1])
    # 唯一主键awj_teacher_id+awjcls_lesson_id check
    cache = df_qc.groupby(['awj_teacher_id', 'awjcls_lesson_id'], 
                                      as_index=False)['score'].count()
    print('df_qc key check_duplicate no(awj_teacher_id, awjcls_lesson_id):', 
          cache.loc[cache['score'] > 1].shape[0], '\n')
    
    ##### 老师行为表 #####
    df_teacher_behavior = pd.read_csv(path + '老师行为信息明细.csv', sep=',', encoding='utf-8')
    # dtypes
    df_teacher_behavior['awj_teacher_id'] = df_teacher_behavior['awj_teacher_id'].astype('int')
    df_teacher_behavior['start_time'] = pd.to_datetime(df_teacher_behavior['start_time'])
    df_teacher_behavior['end_time'] = pd.to_datetime(df_teacher_behavior['end_time'])
    df_teacher_behavior['actual_start_time'] = pd.to_datetime(df_teacher_behavior['actual_start_time'])
    df_teacher_behavior['actual_end_time'] = pd.to_datetime(df_teacher_behavior['actual_end_time'])
    df_teacher_behavior.drop_duplicates(subset=list(df_teacher_behavior.columns), inplace=True)
    # sort
    df_teacher_behavior = df_teacher_behavior.sort_values(by=[
            'awj_teacher_id', 'awj_lesson_id', 'start_time', '积分变化'], ascending=[1, 1, 1, 1])
    print('df_teacher_behavior shape:', df_teacher_behavior.shape)
    # 老师id加lesson_id应该唯一，但有时有重复情况，因为有些老师先请了假，后来又来上课了，所以请假应该去除
    df_teacher_behavior = df_teacher_behavior.groupby(
                ['awj_teacher_id', 'awj_lesson_id'], as_index=False).last()
    print('df_teacher_behavior shape(teacher_id+lesson_id as key:)', df_teacher_behavior.shape)
    # 与monitoring表对比，去除老师abnormal_lesson细分为4，5状态下的课程记录
    df_teacher_behavior = pd.merge(df_teacher_behavior, 
                    df_teacher_monitoring[['awj_teacher_id', 'awjcls_lesson_id', 'abnormal_type']], 
                    left_on=['awj_teacher_id', 'awj_lesson_id'], 
                    right_on=['awj_teacher_id', 'awjcls_lesson_id'], how='left')
    index = df_teacher_behavior.loc[
                    (df_teacher_behavior['teacher_status_for_lesson'] == 'abnormal_lesson') & 
                    (df_teacher_behavior['abnormal_type'].isin([4, 5]))].index
    df_teacher_behavior = df_teacher_behavior.loc[~df_teacher_behavior.index.isin(list(index))]
    df_teacher_behavior.drop(['awjcls_lesson_id', 'abnormal_type'], axis=1, inplace=True)
    print('df_teacher_behavior shape(no abnormal_type=4, 5)', df_teacher_behavior.shape)
    # 唯一主键awj_teacher_id+awjcls_lesson_id check
    cache = df_teacher_behavior.groupby(['awj_teacher_id', 'awj_lesson_id'], 
                                      as_index=False)['start_time'].count()
    print('df_teacher_behavior key check_duplicate no(awj_teacher_id, awj_lesson_id):', 
          cache.loc[cache['start_time'] > 1].shape[0], '\n')
    
    ##### 老师信息表 #####
    df_teacher_info = pd.read_csv(path + '老师基本信息.csv', sep=',', encoding='utf-8')
    # dtypes
    df_teacher_info['awj_teacher_id'] = df_teacher_info['awj_teacher_id'].astype(int)
    df_teacher_info['创建时间'] = pd.to_datetime(df_teacher_info['创建时间'])
    df_teacher_info['首次上架时间'] = pd.to_datetime(df_teacher_info['首次上架时间'])
    df_teacher_info['首课时间'] = pd.to_datetime(df_teacher_info['首课时间'])
    df_teacher_info.drop_duplicates(
        subset=list(df_teacher_info.columns), keep='first', inplace=True)
    # 根据业务要求只取某些type类型老师，其他去除
    df_teacher_info = df_teacher_info.loc[df_teacher_info['teacher_type'].isin([
                'booking&arrangement', 'arrangement_only', 'booking_only'])]
    # sort
    df_teacher_info = df_teacher_info.sort_values(by=['awj_teacher_id'], ascending=[1])
    df_teacher_info = df_teacher_info[['awj_teacher_id', 'state', '创建时间', 
                                                           '首次上架时间', '首课时间']]
    print('df_teacher_info key check duplicate no(awj_teacher_id):', 
          df_teacher_info.shape[0] - df_teacher_info['awj_teacher_id'].nunique(), '\n')
    
    ##### 学生评价明细表 #####
    df_stu_comment = pd.read_csv(path + '学生评价明细.csv', sep=',', encoding='utf-8')
    # dtypes
    df_stu_comment['awj_teacher_id'] = df_stu_comment['awj_teacher_id'].astype(int)
    df_stu_comment['学生评价星级'] = df_stu_comment['学生评价星级'].astype('str')
    df_stu_comment = df_stu_comment.loc[df_stu_comment['学生评价星级'].isin(['1-star', '2-star', 
                                                            '3-star', '4-star', '5-star'])]
    df_stu_comment['评价时间'] = pd.to_datetime(df_stu_comment['评价时间'])
    df_stu_comment['课程开始时间'] = pd.to_datetime(df_stu_comment['课程开始时间'])
    df_stu_comment['课程结束时间'] = pd.to_datetime(df_stu_comment['课程结束时间'])
    df_stu_comment.drop_duplicates(subset=list(df_stu_comment.columns), keep='first', inplace=True)
    # sort
    df_stu_comment = df_stu_comment.sort_values(by=[
                            'awj_teacher_id', '评价时间'], ascending=[1, 1]) 
#     # 老师一次上两堂课，好评不应只算作1次
#     df_stu_comment['lesson_count'] = ((df_stu_comment['课程结束时间'] - 
#                               df_stu_comment['课程开始时间']).dt.total_seconds()) / 3600 * 2
    # 此处仍然算一次，两次效果非常差
    df_stu_comment['lesson_count'] = 1
    print('df_stu_comment before drop duplicates:', df_stu_comment.shape)
    df_stu_comment = df_stu_comment.groupby(['评价id'], as_index=False).last()
    print('df_stu_comment after drop duplicates:', df_stu_comment.shape)
    # 唯一主键awj_teacher_id+awjcls_lesson_id check
    cache = df_stu_comment.groupby(['评价id'], as_index=False)['学生评价星级'].count()
    print('df_stu_comment key check_duplicate no(评价id):', 
                  cache.loc[cache['学生评价星级'] > 1].shape[0], '\n')
    
    # 函数调用
    df_qc_res = teacher_qc_score(start_time, end_time, df_qc, 
                                 class_type_name_special, hq_name_special)
    df_teacher_behavior_res, df_teacher_behavior_processed = teacher_behavior(
                        start_time, end_time, df_teacher_behavior, 
                        class_type_name_special, hq_name_special)
    # 学生评价表中有些课程是floater的课程，需要与行为表课程对比删除（行为表无该情况）
    df_teacher_behavior_res_cache, df_teacher_behavior_processed_cache = teacher_behavior(
                        start_time - DateOffset(years=5), end_time, df_teacher_behavior, 
                        class_type_name_special, hq_name_special)
    df_teacher_behavior_processed_cache = df_teacher_behavior_processed_cache[[
                                        'awj_teacher_id', 'awj_lesson_id']]
    df_teacher_behavior_processed_cache['is_deleted'] = 'no'
    df_stu_comment_res, df_stu_comment_check = stu_comment(start_time, end_time, df_stu_comment, 
                    class_type_name_special, hq_name_special, df_teacher_behavior_processed_cache)
    # 宽表
    df_wide = pd.merge(df_teacher_info, df_teacher_behavior_res, on='awj_teacher_id', how='left')
    df_wide = pd.merge(df_wide, df_qc_res, on='awj_teacher_id', how='left')
    df_wide = pd.merge(df_wide, df_stu_comment_res, on='awj_teacher_id', how='left')
    # 有些老师3个月内可能无qc分数，所以取历史qc记录处理后填补0分情况
    df_qc_all = teacher_qc_score(start_time - DateOffset(years=5), end_time, df_qc, 
                                             class_type_name_special, hq_name_special)
    df_qc_all = df_qc_all[['awj_teacher_id', 'teacher_score_max', 
                               'teacher_score_min', 'log_decay_score_mean']]
    df_qc_all.rename(columns={'teacher_score_max': 'teacher_score_max_all', 
                         'teacher_score_min': 'teacher_score_min_all', 
                         'log_decay_score_mean': 'log_decay_score_mean_all'}, inplace=True)
    df_wide = pd.merge(df_wide, df_qc_all, on='awj_teacher_id', how='left')
    # 有些老师没上过课，但请过假，此处为了让这些老师参加评分而非直接三星，所以将normal_lesson_log_count平滑处理
    df_wide.loc[(df_wide['normal_lesson_log_count'] == 0) & 
            (df_wide['ask_for_leave_log_count'] > 0), 'normal_lesson_log_count'] = math.log(2, 10)
    # 衍生新字段
    df_wide['log_ask_for_leave/log_normal_lesson'] = df_wide['ask_for_leave_log_count'] / df_wide['normal_lesson_log_count']
    df_wide['abnormal_all_log_count'] = (df_wide['no_show_lesson_log_count'] + 
                                      df_wide['late_lesson_log_count'] + 
                                      df_wide['abnormal_lesson_log_count'] + 
                                      df_wide['ask_for_leave_log_count'])
    df_wide['abnormal_all_log_percent'] = (df_wide['abnormal_all_log_count']) / (
                                      df_wide['normal_lesson_log_count'] + 
                                      df_wide['abnormal_all_log_count'])
    df_wide['lesson_time_range'] = ((end_time - df_wide['首课时间']).dt.total_seconds()) / (3600 * 24) 
    df_wide.loc[df_wide['lesson_time_range'] == 0, 
        'lesson_time_range'] = ((end_time - df_wide['创建时间']).dt.total_seconds()) / (3600 * 24)  
    # 判断是否是新老师
    df_wide['old_new_teacher'] = 'old'
    df_wide.loc[(df_wide['首课时间'].isnull()) & 
                (df_wide['state'].isin(['oboard', 'active'])), 'old_new_teacher'] = 'new'
    # 缺失值填补
    # 无用字段去除
    df_wide.drop(['创建时间', '首次上架时间', '首课时间', 'state'], axis=1, inplace=True)
    columns = list(df_wide.columns)
    columns.pop(columns.index('awj_teacher_id'))
    columns.pop(columns.index('old_new_teacher'))
    # new teacher: mean
    for itm in columns:
        df_wide.loc[df_wide['old_new_teacher'] == 'new', itm] = df_wide.loc[
            (df_wide['normal_lesson_log_count'] > 0), itm].mean()
    # old teacher:0
    df_wide.fillna(value=0, inplace=True)
    new_teacher = df_wide.loc[df_wide['old_new_teacher'] == 'new', 'awj_teacher_id']
    print('new teacher no:', new_teacher.shape[0])
    # 有些老师没有请过假，advanced_days字段为0，填为均值
    df_wide.loc[(df_wide['ask_for_leave_log_count'] == 0) & (df_wide['normal_lesson_log_count'] > 0), 
        'advanced_days_max'] = df_wide.loc[(df_wide['advanced_days_max'] != 0) & 
                        df_wide['normal_lesson_log_count'] > 0, 'advanced_days_max'].mean()
    df_wide.loc[(df_wide['ask_for_leave_log_count'] == 0) & (df_wide['normal_lesson_log_count'] > 0), 
        'advanced_days_log_mean'] = df_wide.loc[(df_wide['advanced_days_log_mean'] != 0) & 
                        df_wide['normal_lesson_log_count'] > 0, 'advanced_days_log_mean'].mean()
    # 平滑
    smooth = ['late_lesson_log_count', 'no_show_lesson_log_count', 'abnormal_lesson_log_count', 
              'ask_for_leave_log_count', 
              'stu_comment_log_bad_behavior']
    for itm in smooth:
        df_wide[itm] = df_wide[itm] + math.log(2, 10)
    # 字段处理
    # 比例计算
    df_wide['late_lesson_log_percent'] = df_wide['late_lesson_log_count'] / df_wide['normal_lesson_log_count']
    df_wide['no_show_lesson_log_percent'] = df_wide['no_show_lesson_log_count'] / df_wide['normal_lesson_log_count']
    df_wide['abnormal_lesson_log_percent'] = df_wide['abnormal_lesson_log_count'] / df_wide['normal_lesson_log_count']
    df_wide['ask_for_leave_log_percent'] = df_wide['ask_for_leave_log_count'] / df_wide['normal_lesson_log_count']
    # 查看上课异常行为的分位数
    columns = ['no_show_lesson_log_percent', 'late_lesson_log_percent', 
               'abnormal_lesson_log_percent', 'ask_for_leave_log_percent', 
               'abnormal_all_log_percent', 'log_ask_for_leave/log_normal_lesson', 
               'advanced_days_log_mean', 'advanced_days_max']
    quantiles = [0.1, 0.15, 0.2, 0.25, 0.3, 0.33, 0.5, 0.55, 0.6, 0.66, 0.7, 0.75, 
                 0.77, 0.8, 0.83, 0.85, 0.9, 0.92, 0.95, 1]
    for i in range(len(columns)):
        print('\n', columns[i], ':')
        for itm in quantiles:
            print(str(itm), ':', df_wide.loc[df_wide['normal_lesson_log_count'] > 0, 
                                                        columns[i]].quantile(itm))
    # 老师异常行为有一项出现较大异常值时或整体较差，normal_lesson_log_count降为相应较低数值，整体表现变差
    counts = ['ask_for_leave_log_count', 'late_lesson_log_count', 
               'no_show_lesson_log_count', 'abnormal_lesson_log_count', 
               'abnormal_all_log_count']
    columns = ['ask_for_leave_log_percent', 'late_lesson_log_percent', 
               'no_show_lesson_log_percent', 'abnormal_lesson_log_percent', 
               'abnormal_all_log_percent']
    cache_columns = ['normal_lesson_log_count_processed_ask_for_leave_cahce1', 
                     'normal_lesson_log_count_processed_late_lesson_cache2', 
                     'normal_lesson_log_count_processed_no_show_cache3', 
                     'normal_lesson_log_count_processed_abnormal_lesson_cache4', 
                     'normal_lesson_log_count_processed_abnormal_all_cahce5'
                    ]
    for itm in cache_columns:
        df_wide[itm] = df_wide['normal_lesson_log_count']
    df_wide['normal_lesson_log_count_processed'] = df_wide['normal_lesson_log_count']
    # 分位数
    quantiles = [[0.6, 0.7, 0.75, 0.8, 1], [0.8, 0.9, 0.92, 0.95, 1], 
                [0.5, 0.6, 0.66, 0.75, 0.8, 0.85, 0.9, 1], [0.8, 0.85, 0.9, 0.95, 1], 
                [0.55, 0.66, 0.75, 0.85, 1]]
    # 降低比例
    indexes = [[0.8, 0.6, 0.3, 0.1], [0.8, 0.7, 0.3, 0.1], 
              [0.8, 0.75, 0.6, 0.5, 0.3, 0.1, 0.03], [0.9, 0.85, 0.7, 0.3], 
              [0.8, 0.7, 0.3, 0.1]]
    for i in range(len(columns)):
        for k in range(len(quantiles[i]) - 1):
            standard1 = df_wide.loc[df_wide['normal_lesson_log_count'] > 0, 
                                    columns[i]].quantile(quantiles[i][k])
            standard2 = df_wide.loc[df_wide['normal_lesson_log_count'] > 0, 
                                    columns[i]].quantile(quantiles[i][k + 1])
            # 降低正常上课的数量
            df_wide.loc[(df_wide['normal_lesson_log_count'] > 0) & 
                (df_wide[columns[i]] > standard1) & (df_wide[columns[i]] <= standard2), 
                cache_columns[i]] = df_wide['normal_lesson_log_count'] * indexes[i][k]
            # 新老师因为平滑原因，所以再复原，不惩罚
            df_wide.loc[(df_wide['normal_lesson_log_count'] > 0) & 
                (df_wide[columns[i]] > standard1) & (df_wide[columns[i]] <= standard2) & 
                (df_wide[counts[i]] == math.log(2, 10)) & 
                (df_wide['normal_lesson_log_count'] <= 8), 
                cache_columns[i]] = df_wide['normal_lesson_log_count']
    # 从5行cache_colume中取最小值
    df_wide['normal_lesson_log_count_processed'] = df_wide[cache_columns].min(axis=1)
    # 大小方向统一化
    # 重新计算四个percent
    df_wide['late_lesson_log_percent_processed'] = df_wide['late_lesson_log_count'] / df_wide['normal_lesson_log_count_processed']
    df_wide['no_show_lesson_log_percent_processed'] = df_wide['no_show_lesson_log_count'] / df_wide['normal_lesson_log_count_processed']
    df_wide['abnormal_lesson_log_percent_processed'] = df_wide['abnormal_lesson_log_count'] / df_wide['normal_lesson_log_count_processed']
    df_wide['ask_for_leave_log_percent_processed'] = df_wide['ask_for_leave_log_count'] / df_wide['normal_lesson_log_count_processed']
    df_wide['normal_log_lesson_per_week'] = df_wide['normal_lesson_log_count_processed'] / (df_wide['lesson_time_range']) 
    # 取倒数
    rcp = ['late_lesson_log_processed', 'no_show_lesson_log_processed', 
           'abnormal_lesson_log_processed', 'ask_for_leave_log_processed', 
           'stu_comment_log_bad_behavior_processed']
    cols = ['late_lesson_log_percent_processed', 'no_show_lesson_log_percent_processed', 
            'abnormal_lesson_log_percent_processed', 'ask_for_leave_log_percent_processed', 
            'stu_comment_log_bad_behavior']
    for i in range(len(rcp)):
        df_wide[rcp[i]] = 1 / df_wide[cols[i]]
    # 修正老师上课数量少但好评较多的情况（如老师id642）
    columns = ['late_lesson_log_count', 'no_show_lesson_log_count', 
              'ask_for_leave_log_count', 'abnormal_lesson_log_count']
    no_smooths = ['late_lesson_log_count_no_smooth', 'no_show_lesson_log_count_no_smooth', 
              'ask_for_leave_log_count_no_smooth', 'abnormal_lesson_log_count_no_smooth']
    for i in range(len(columns)):
        df_wide[no_smooths[i]] = df_wide[columns[i]]
        df_wide.loc[df_wide[no_smooths[i]] == math.log(2, 10), no_smooths[i]] = 0
    df_wide['stu_comment_log_good_behavior_processed'] = df_wide['stu_comment_log_good_behavior'] / ( 
                                df_wide['normal_lesson_log_count']
                                + df_wide['late_lesson_log_count_no_smooth'] 
                                + df_wide['no_show_lesson_log_count_no_smooth'] 
                                + df_wide['ask_for_leave_log_count_no_smooth'] 
                                + df_wide['abnormal_lesson_log_count_no_smooth'])                    
    df_wide['stu_comment_log_bad_behavior_processed'] = df_wide['stu_comment_log_bad_behavior'] / ( 
                                df_wide['normal_lesson_log_count'])
    # 修正老师请假次数过多，但提前请假天数指标过好的情况（如老师id642） 
    columns = ['advanced_days_log_mean', 'advanced_days_max']
    quantiles = [0.6, 0.66, 0.7, 0.75, 0.77, 0.8, 0.85, 1]
    indexes = [0.9, 0.8, 0.7, 0.5, 0.3, 0.1, 0.05]
    for i in range(len(columns)):
        for k in range(len(quantiles) - 1):
            percent1 = df_wide.loc[df_wide['normal_lesson_log_count'] > 0, 
                                'log_ask_for_leave/log_normal_lesson'].quantile(quantiles[k])
            percent2 = df_wide.loc[df_wide['normal_lesson_log_count'] > 0, 
                                'log_ask_for_leave/log_normal_lesson'].quantile(quantiles[k + 1])
            df_wide.loc[(df_wide['normal_lesson_log_count'] > 0) & 
                        (df_wide['log_ask_for_leave/log_normal_lesson'] > percent1) & 
                        (df_wide['log_ask_for_leave/log_normal_lesson'] <= percent2), 
                        columns[i]] = df_wide[columns[i]] * indexes[k]
    # 修正有些老师过去3个月内无qc，分数为0分的情况
    columns = ['log_decay_score_mean', 'teacher_score_min', 'teacher_score_max']
    for itm in columns:
        df_wide.loc[(df_wide['normal_lesson_log_count'] > 0) & 
                   (df_wide[itm].isin([np.nan, 0])), itm] = df_wide[itm + '_all']
#     # 修正有些老师过去三个月内无评价星级的情况-该指标没有用
#     print('no of teacher with no stu comment star rate:', 
#           df_wide.loc[(df_wide['normal_lesson_log_count'] > 0) & 
#         (df_wide['stu_log_star'].isin([np.nan, 0])), 'stu_log_star'].shape[0])
#     df_wide.loc[(df_wide['normal_lesson_log_count'] > 0) & 
#         (df_wide['stu_log_star'].isin([np.nan, 0])), 'stu_log_star'] = df_wide.loc[
#         (df_wide['normal_lesson_log_count'] > 0) & 
#         (~df_wide['stu_log_star'].isin(['no.nan, 0'])), 'stu_log_star'].median()
    # 无上课记录老师
    columns = list(df_wide.columns)
    columns.pop(columns.index('awj_teacher_id'))
    for itm in columns:
        df_wide.loc[df_wide['normal_lesson_log_count'] == 0, itm] = 0
    # 只取有行为数据的
    df_wide_final = df_wide.loc[df_wide['normal_lesson_log_count'] > 0]
    # delete columns
    df_wide_final.drop(['advanced_days_std', 'teacher_score_std', 
                  'advanced_days_min', 'lesson_time_range', 'old_new_teacher', 
                  'teacher_qc_count', 'normal_lesson_log_count', 'late_lesson_log_count', 
                  'no_show_lesson_log_count', 'abnormal_lesson_log_count', 
                  'ask_for_leave_log_count', 'normal_log_lesson_per_week', 
                  'stu_comment_log_good_behavior', 'stu_comment_log_bad_behavior', 
                  'log_ask_for_leave/log_normal_lesson',  'abnormal_lesson_log_percent',
                  'late_lesson_log_percent', 'no_show_lesson_log_percent', 
                  'ask_for_leave_log_percent', 'abnormal_all_log_count', 
                  'late_lesson_log_percent_processed', 'no_show_lesson_log_percent_processed', 
                  'abnormal_lesson_log_percent_processed', 'ask_for_leave_log_percent_processed', 
                  'normal_lesson_log_count_processed', 'abnormal_all_log_percent', 
                  'normal_lesson_log_count_processed_ask_for_leave_cahce1', 
                  'normal_lesson_log_count_processed_late_lesson_cache2', 
                  'normal_lesson_log_count_processed_no_show_cache3', 
                  'normal_lesson_log_count_processed_abnormal_lesson_cache4', 
                  'normal_lesson_log_count_processed_abnormal_all_cahce5',   
                  'log_decay_score_mean_all', 'teacher_score_max_all', 'teacher_score_min_all', 
                  'stu_comment_log_bad_behavior_processed', 
                  'late_lesson_log_count_no_smooth', 'no_show_lesson_log_count_no_smooth', 
                  'ask_for_leave_log_count_no_smooth', 'abnormal_lesson_log_count_no_smooth'], axis=1, inplace=True)
    # save
    df_wide_final.to_csv(path + 'df_wide_log_final.csv', sep=',', float_format='%.5f', 
                         index=False, encoding='utf-8')

start_time: 2018-01-30 23:59:59 
 end_time: 2018-04-30 23:59:59 

df_teacher_monitoring key check_duplicate no(awj_teacher_id, awjcls_lesson_id): 0 

df_qc_before drop duplicates: (29311, 17)
df_qc_drop duplicates: (24184, 17)
df_qc key check_duplicate no(awj_teacher_id, awjcls_lesson_id): 0 

df_teacher_behavior shape: (241621, 13)
df_teacher_behavior shape(teacher_id+lesson_id as key:) (241469, 13)
df_teacher_behavior shape(no abnormal_type=4, 5) (241467, 13)
df_teacher_behavior key check_duplicate no(awj_teacher_id, awj_lesson_id): 0 

df_teacher_info key check duplicate no(awj_teacher_id): 0 

df_stu_comment before drop duplicates: (329911, 13)
df_stu_comment after drop duplicates: (99008, 13)
df_stu_comment key check_duplicate no(评价id): 0 

df_stu_comment_floater classes existed: (28023, 13)
df_stu_comment_floater classes deleted: (27203, 13)
new teacher no: 34

 no_show_lesson_log_percent :
0.1 : 0.0018819989318836902
0.15 : 0.0025431068958906575
0.2 : 0.0031769821761364744
0.25 

In [22]:
# FA_indexs
fa_index = pd.read_excel(path + 'fa_indexs.xlsx')
fa_index

,fa_index1,fa_index2,fa_index3,fa_index4
0,-0.062063,-0.028543,0.530873,0.008978
1,-0.087088,-0.017771,0.545701,-0.019414
2,-0.027714,0.344296,-0.032156,0.028113
3,-0.036995,0.346024,-0.001726,-0.060828
4,-0.032262,0.358875,-0.021149,-0.026614
5,0.294944,-0.023287,-0.051459,-0.005215
6,0.324883,-0.028596,-0.071127,0.020754
7,0.321721,-0.017894,-0.087450,0.007256
8,0.275466,-0.043566,-0.015902,0.034857
9,0.023129,-0.028920,-0.006686,1.000274


In [23]:
fa_index = fa_index.as_matrix()
df_wide_matrix = deepcopy(df_wide_final)
df_wide_matrix.drop(['awj_teacher_id'], axis=1, inplace=True)
df_wide_matrix = df_wide_matrix.as_matrix()
# df_wide标准化
df_wide_matrix = preprocessing.scale(df_wide_matrix)
# 每个老师的各因子得分
fa_score = np.dot(df_wide_matrix, fa_index)
# 主成分贡献率
var = np.array([[0.39025 / 0.87332], 
                [0.23851 / 0.87332], 
                [0.14904 / 0.87332], 
                [0.09552 / 0.87332]])
# 每个老师的最终得分
final_score = np.dot(fa_score, var)

In [25]:
# df格式
teacher_fa_score = np.hstack((fa_score, final_score))
teacher_fa_score = pd.DataFrame(teacher_fa_score)
teacher_fa_score['awj_teacher_id'] = list(df_wide_final['awj_teacher_id'])
teacher_fa_score.rename(columns={0: 'teacher_behavior_score', 1: 'teacher_qc_score', 
                                 2:'teacher_attitude_score', 3: 'student_comment_score', 
                                 4: 'final_score'}, inplace=True)
# 星级映射(去除过去一段时间周期内没上过课的老师)
teacher_fa_score = teacher_fa_score.sort_values(by='final_score', ascending=0)
# 业务要求的分位数
star_5 = teacher_fa_score['final_score'].quantile(0.8)
star_4 = teacher_fa_score['final_score'].quantile(0.5)
star_3 = teacher_fa_score['final_score'].quantile(0.2)
star_2 = teacher_fa_score['final_score'].quantile(0.1)
print('5:', star_5, '4:', star_4, '3:', star_3, '2:', star_2)
teacher_fa_score.loc[teacher_fa_score['final_score'] <= star_2, 'star'] = 1
teacher_fa_score.loc[(teacher_fa_score['final_score'] > star_2) & 
                     (teacher_fa_score['final_score'] <= star_3), 'star'] = 2
teacher_fa_score.loc[(teacher_fa_score['final_score'] > star_3) & 
                     (teacher_fa_score['final_score'] <= star_4), 'star'] = 3
teacher_fa_score.loc[(teacher_fa_score['final_score'] > star_4) & 
                     (teacher_fa_score['final_score'] <= star_5), 'star'] = 4
teacher_fa_score.loc[teacher_fa_score['final_score'] > star_5, 'star'] = 5

teacher_fa_score.describe()

5: 0.4435888982487725 4: -0.11104762421495835 3: -0.4447382937613706 2: -0.5616948927524096


,teacher_behavior_score,teacher_qc_score,teacher_attitude_score,student_comment_score,final_score,awj_teacher_id,star
count,5.390000e+02,5.390000e+02,5.390000e+02,5.390000e+02,5.390000e+02,539.000000,539.000000
mean,4.891856e-16,-3.015522e-16,1.120522e-16,1.660185e-16,2.068022e-16,1177.402597,3.398887
std,1.010837e+00,9.970982e-01,9.996157e-01,1.001465e+00,6.115678e-01,552.708329,1.201950
min,-1.103754e+00,-7.017688e+00,-1.660816e+00,-1.590315e+00,-2.163993e+00,54.000000,1.000000
25%,-7.289620e-01,-3.559515e-02,-8.904390e-01,-8.205419e-01,-3.768955e-01,681.500000,3.000000
50%,-4.016600e-01,1.275854e-01,-5.261013e-02,6.716698e-02,-1.110476e-01,1219.000000,3.000000
75%,3.930729e-01,2.867032e-01,7.096040e-01,5.880385e-01,3.197617e-01,1712.500000,4.000000
max,4.704800e+00,1.319065e+00,4.933427e+00,4.695286e+00,2.265276e+00,1997.000000,5.000000


In [14]:
# 拼回去
teacher_fa_score = pd.merge(teacher_fa_score, df_wide[
    ['awj_teacher_id', 'normal_lesson_log_count']], on='awj_teacher_id', how='right')
teacher_fa_score.fillna(value=0, inplace=True)
teacher_fa_score.to_csv(path + 'teacher_star.csv', sep=',', 
                        float_format='%.5f', encoding='utf-8', index=False)
teacher_fa_score

,teacher_behavior_score,teacher_qc_score,teacher_attitude_score,student_comment_score,final_score,awj_teacher_id,star,normal_lesson_log_count
0,4.704800,0.302857,-0.244668,1.114883,2.265276,871,5.0,314.655480
1,4.096161,0.403369,-0.137052,0.921520,2.017968,279,5.0,281.496993
2,4.195220,-0.071245,-0.131346,0.966546,1.938511,1287,5.0,338.831899
3,4.089784,0.172417,-0.113986,0.188793,1.875837,799,5.0,280.835440
4,4.253281,-0.078318,-0.126928,0.057958,1.863901,508,5.0,288.811278
5,3.894182,0.002799,-0.079544,1.161929,1.854423,317,5.0,336.231561
6,3.240825,0.149709,0.045204,1.383593,1.648121,1066,5.0,232.142592
7,2.716653,0.324876,0.136515,1.210233,1.458351,1130,5.0,204.106830
8,2.938710,-0.041000,0.166677,0.888884,1.427656,1141,5.0,281.440194
9,3.233572,0.190346,-0.925128,0.801442,1.426709,390,5.0,273.051470
